### Imports

In [1]:
import interflow

In [35]:
df = interflow.calc_pws_commercial_industrial_flows()

In [23]:
output = interflow.prep_public_water_supply_fraction()
FIPS = '56037'
result = output[output.FIPS == FIPS]
#result = result['com_pws_fraction'][1]
result['com_pws_fraction'].iloc[0]

0.34976525821596244

In [36]:
import plotly.figure_factory as ff

hist_data = [df['IND_public_total_total_total_mgd_from_PWD_total_total_total_total_mgd'].to_list()]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, show_hist=False)
fig.show()

In [65]:
df = interflow.get_electricity_demand_data()
df_county = interflow.rename_water_data_2015(variables=['FIPS', 'State', 'County', 'population'])
x = df_county['population'][0]

df_state = df_county.groupby("State", as_index=False).sum()
s = df_state[df_state.State == 'AL']
s = s['population'][1]

pop = x/s


pop

0.011390664582003749

In [32]:
df = interflow.get_electricity_demand_data()
df = df[df.Year == 2015]
df = df[df.State != 'US']
df = df.loc[df['Industry Sector Category'] == 'Total Electric Industry']

out = interflow.calc_population_county_weight(df)
out

,Year,State,Industry Sector Category,Residential,Commercial,Industrial,Transportation,Other,Total,FIPS,County,pop_weight
0,2015,AK,Total Electric Industry,2044184.0,2763491.0,1351529.0,0.0,NaN,6159204,02013,Aleutians East Borough,0.004524
1,2015,AK,Total Electric Industry,2044184.0,2763491.0,1351529.0,0.0,NaN,6159204,02016,Aleutians West Census Area,0.007722
2,2015,AK,Total Electric Industry,2044184.0,2763491.0,1351529.0,0.0,NaN,6159204,02020,Municipality of Anchorage,0.404499
3,2015,AK,Total Electric Industry,2044184.0,2763491.0,1351529.0,0.0,NaN,6159204,02050,Bethel Census Area,0.024303
4,2015,AK,Total Electric Industry,2044184.0,2763491.0,1351529.0,0.0,NaN,6159204,02060,Bristol Bay Borough,0.001208
...,...,...,...,...,...,...,...,...,...,...,...,...
3137,2015,WY,Total Electric Industry,2676702.0,3924590.0,10323470.0,0.0,NaN,16924762,56037,Sweetwater County,0.076140
3138,2015,WY,Total Electric Industry,2676702.0,3924590.0,10323470.0,0.0,NaN,16924762,56039,Teton County,0.039455
3139,2015,WY,Total Electric Industry,2676702.0,3924590.0,10323470.0,0.0,NaN,16924762,56041,Uinta County,0.035526
3140,2015,WY,Total Electric Industry,2676702.0,3924590.0,10323470.0,0.0,NaN,16924762,56043,Washakie County,0.014209


In [19]:
data = flow.read_sample_data()
region = '01001'
df_in = flow.calculate(data=data, region_name=region, level=5)
df

NameError: name 'flow' is not defined

In [ ]:
df = flow.group_results(df=df_in, output_level=1)
output_level1 = flow.group_results(df=df_in, output_level=1)
output_level1 = output_level1[output_level1.T1 == 'CMP']
output_level1 = output_level1[output_level1.S1 == 'AGR']
x_1 = output_level1['value'].sum()
x_1

In [ ]:
df = flow.prep_electricity_fuel()
df = df[df.FIPS == '36061']
df

In [ ]:
df = flow.prep_electricity_cooling()
df = df[df.plant_code == 2493]
#df = df['EGS_natgas_combustionturbine_oncethrough_total_mgd_from_WSW_saline_surfacewater_total_total_mgd']
df

## Load Input Data

In [ ]:
%%time
#input_data = r"C:\Users\mong275\Local Files\Repos\flow\sample_data\test_output.csv"
data = interflow.read_sample_data()
data

## Run the calculations for a single US county or all counties

Wall time: 1.3 s


,region,S1,S2,S3,S4,S5,T1,T2,T3,T4,T5,units,value
0,01003,WSW,fresh,groundwater,total,total,AGR,aquaculture,fresh,groundwater,withdrawal,mgd,0.050000
1,01003,WSW,fresh,surfacewater,total,total,AGR,aquaculture,fresh,surfacewater,withdrawal,mgd,0.050000
2,01003,WSW,fresh,groundwater,total,total,AGR,crop,fresh,groundwater,withdrawal,mgd,47.113532
3,01003,WSW,fresh,surfacewater,total,total,AGR,crop,fresh,surfacewater,withdrawal,mgd,6.652714
4,01003,WSW,fresh,groundwater,total,total,AGR,ethanol,fresh,groundwater,withdrawal,mgd,0.056468
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,01003,WWD,secondary,municipal,total,total,SRD,total,total,total,total,mgd,0.764113
173,01003,WWD,treatment,advanced,total,total,ESV,total,total,total,total,bbtu,0.003390
174,01003,WWD,treatment,advanced,total,total,REJ,total,total,total,total,bbtu,0.001825
175,01003,WWD,treatment,secondary,total,total,ESV,total,total,total,total,bbtu,0.004808


In [27]:
region = '45067'
df = interflow.calculate(data = data, region_name=region)#, output_file_path=r"C:\Users\mong275\Downloads\output.csv")

In [28]:
df.replace("WSW", "Water Supply", inplace=True)
df.replace("AGR", "Agriculture", inplace=True)
df.replace("RES", "Residential", inplace=True)
df.replace("IND", "Industrial", inplace=True)
df.replace("PWD", "Public Water Demand", inplace=True)
df.replace("PWI", "Public Water Imports", inplace=True)
df.replace("COM", "Commercial", inplace=True)
df.replace("WWS", "Wastewater Supply", inplace=True)
df.replace("WWD", "Wastewater Treatment", inplace=True)
df.replace("CMP", "Consumption/Evaporation", inplace=True)
df.replace("GWD", "Ground Discharge", inplace=True)
df.replace("SRD", "Surface Discharge", inplace=True)
df.replace("IRX", "Discharge to Irrigation", inplace=True)
df.replace("INX", "Discharge to Industrial", inplace=True)
df.replace("OCD", "Discharge to Ocean", inplace=True)
df.replace("PWS", "Public Water Supply", inplace=True)
df.replace("EPD", "Energy Demand", inplace=True)
df.replace("EGD", "Electricity Demand", inplace=True)

df.replace("TRA", "Transportation", inplace=True)
df.replace("ESV", "Energy Services", inplace=True)
df.replace("REJ", "Rejected Energy", inplace=True)

## Produce Water & Energy Sankey Diagrams for a County

In [29]:
%%time


viz = interflow.plot_sankey(data=df, region_name= region,
                       unit_type1 = 'mgd', unit_type2='bbtu', output_level=2, strip='total')

Wall time: 33.9 ms


## Produce Sector Barcharts for a County

In [ ]:
sectors = ['PWS', 'AGR']
flow.plot_sector_bar(data=df, unit_type='bbtu', region_name =region, sector_list=sectors, inflow=True, strip='total')

## County Map Visualization for all Values
Note that this diagram requires the full run output (all counties) if you want to see a map of all counties. Otherwise it will just display the county it has data for.

In [ ]:
import pandas as pd
import flow  

# read in full output dataset
df = pd.read_csv(r"C:\Users\mong275\Local Files\Repos\full_run_02-25.csv", dtype={'region':str})
# add state and county columns
#df['countyname'] = 'countyname'

# rearrange column order
#col_list = df.columns[1:-1].to_list()
#df = df[['region', 'countyname'] + col_list]

flow.plot_map(data=df, 
         #region_col=1,
         jsonfile = geo,
             level=2)

In [ ]:
maps = pd.read_csv(r"C:\Users\mong275\Local Files\Repos\flow\sample_data\input_data\county_interconnect_list.csv", dtype={'FIPS':str})
maps['FIPS'] = maps['FIPS'].apply(lambda x: '{0:0>5}'.format(x))
maps = maps.rename(columns = {'FIPS':'region'})
full = pd.merge(df, maps, how='left', on='region')
full.reset_index()
full = full.rename_axis(None, axis=1)                           #drop index name
full.index.is_unique
full

In [ ]:
group_list = ['STATE','S1','S2','S3','S4','S5','T1','T2','T3','T4','T5','units']
state = full.drop(['region'], axis=1)
state = state.groupby(group_list, as_index=False).sum()
state = state.rename(columns={'STATE':'region'})
#state.reset_index()
#state.index.is_unique
state

In [ ]:
viz = flow.plot_sankey(data=state, region_name="NJ", 
                       unit_type1 = 'mgd', unit_type2='bbtu', output_level=1, strip='total', remove_sectors=['EGS'])

In [ ]:
full = pd.read_csv(r"C:\Users\mong275\Local Files\Repos\full_run_output.csv", dtype={'region':str})
maps = pd.read_csv(r'C:\Users\mong275\Local Files\Repos\flow\flow\data\county_interconnect_list.csv', dtype={'FIPS':str})
maps['FIPS'] = maps['FIPS'].apply(lambda x: '{0:0>5}'.format(x))
maps = maps.rename(columns = {'FIPS':'region'})
full = pd.merge(full, maps, how='left', on='region')
full.reset_index()
full = full.rename_axis(None, axis=1)                           #drop index name
full.index.is_unique
full

In [ ]:
group_list = ['Interconnect','S1','S2','S3','S4','S5','T1','T2','T3','T4','T5','units']
interconnect = full.drop(['region'], axis=1)
interconnect = interconnect.groupby(group_list, as_index=False).sum()
interconnect = interconnect.rename(columns={'Interconnect':'region'})
#state.reset_index()
#state.index.is_unique
interconnect

In [ ]:
viz = flow.plot_sankey(data=interconnect, region_name="West", 
                       unit_type1 = 'mgd', unit_type2='bbtu', output_level=2, strip='total')

In [ ]:
df.to_csv(r"C:\Users\mong275\Local Files\Repos\full_run_02-25.csv", index=False)

In [ ]:
#state = state[state.value < state.value.quantile(.99)]
subset = ['EGS']
state = state[~state['S1'].isin(subset)]
state = state[~state['T1'].isin(subset)]

state

In [ ]:
interconnect

In [ ]:
viz = flow.plot_sankey(data=df, region_name=region, 
                       unit_type1 = 'mgd', unit_type2='bbtu', output_level=3, strip='total')

In [ ]:
full = pd.read_csv(r"C:\Users\mong275\Local Files\Repos\full_run_output.csv")
full['region'] = full['region'].apply(lambda x: '{0:0>5}'.format(x))  # add leading zero to FIPS code


In [ ]:
from urllib.request import urlopen
import json
json_file = r"C:\Users\mong275\Local Files\Repos\flow\flow\data\geojson-counties-fips.json"
f = open(json_file)
counties = json.load(f)
import plotly.express as px


fig = px.choropleth_mapbox(full, geojson=counties, locations='region', color='value',
                           color_continuous_scale="Viridis",
                           #range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
full_copy['one'] = 1

In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import urllib.request

jdata = counties

mycustomdata = full_copy

title = 'Update method called for a  choroplethmapbox chart'

fig = go.Figure(go.Choroplethmapbox(geojson=jdata, 
                                    locations=full_copy['region'], 
                                    z=full_copy['one'],
                                    featureidkey='properties.id',
                                    coloraxis="coloraxis",
                                    customdata=mycustomdata,
                                    marker_opacity=0.75, marker_line_width=0.5))
fig.update_layout(coloraxis_colorscale='Viridis',
                  mapbox=dict(style='carto-positron',
                              zoom=2, 
                              center = {"lat": 37.0902, "lon": -95.7129},
                              )) 
fig.update_layout(title_text=title,
                  title_x=0.5,
                  margin={"r":10,"t":60,"l":0,"b":0});

fig.show()

In [ ]:
matter_r= [[0.0, '#2f0f3d'], #cmocean colorscale 
           [0.1, '#4f1552'],
           [0.2, '#72195f'],
           [0.3, '#931f63'],
           [0.4, '#b32e5e'],
           [0.5, '#cf4456'],
           [0.6, '#e26152'],
           [0.7, '#ee845d'],
           [0.8, '#f5a672'],
           [0.9, '#faca8f'],
           [1.0, '#fdedb0']]



button1 = dict(method= 'update',
               label='2018',
               args=[
                    {"z": [df['2018']],
                     "hovertemplate": "Canton: %{customdata[0]}<br>"+\
                                      "2018: %{customdata[1]}%<extra></extra>"
                    }, #dict for fig.data[0] updates
                    {"coloraxis.colorscale":"Viridis" } #dict for  layout attribute update 
                 ])

button2 = dict(method= 'update',
               label='2019',
               args=[
                   {"z": [df['2019']],
                    "hovertemplate": "Canton: %{customdata[0]}<br>"+\
                                     "2019: %{customdata[2]}%<extra></extra>"
                   },
                   {"coloraxis.colorscale": matter_r} #update layout attribute    
           ])

fig.update_layout(updatemenus=[dict(active=0,
                                    buttons= [button1, button2])]
                                     );

fig.show()

In [ ]:
full_copy = pd.pivot_table(full, values='value', index=['region'],
                               columns=['S1'],
                               aggfunc=np.sum)  # pivot to get discharge types as columns
full_copy = full_copy.reset_index()  # reset index to remove multi-index from pivot table
full_copy = full_copy.rename_axis(None, axis=1)  # drop index name
full_copy.fillna(0, inplace=True)
full_copy

In [ ]:
import json
json_file = r"C:\Users\mong275\Local Files\Repos\flow\flow\data\geojson-counties-fips.json"
f = open(json_file)
counties = json.load(f)
import plotly.express as px

df = pd.read_csv(r"C:\Users\mong275\Local Files\Repos\full_run_output.csv", dtype={'region':str})
df['SOURCE'] = df['S1']+'-'+ df['S2']+'-'+ df['S3']+'-'+ df['S4']+'-'+ df['S5']
df['TARGET'] = df['T1']+'-'+ df['T2']+'-'+ df['T3']+'-'+ df['T4']+'-'+ df['T5']

remove = '-total'
df['SOURCE'] = df['SOURCE'].str.replace(remove, "")
df['TARGET'] = df['TARGET'].str.replace(remove, "")
df['Link'] = df['SOURCE'] +  ' to ' + df['TARGET']+ ', ' +df['units']
df

In [ ]:
df = pd.pivot_table(df, values='value', index=['region'],
                    columns=['Link'], aggfunc=np.sum)

df = df.reset_index()                                       #reset index to remove multi-index from pivot table
df = df.rename_axis(None, axis=1)                           #drop index name

df.fillna(0,inplace=True)

## Code below works

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties, 
                                    locations=df['region'], 
                                    z=df['AAQ-fresh-surfacewater-withdrawal to SRD, mgd'],
                                    #featureidkey='properties.id',
                                    coloraxis="coloraxis",
                                    #customdata=mycustomdata,
                                    #hovertemplate= 'Canton: %{customdata[0]}'+\
                                    #               '<br>2018: %{customdata[1]}%<extra></extra>',
                                    marker_opacity=0.75, 
                                    marker_line_width=0.5))
fig.update_layout(coloraxis_colorscale='Viridis',
                  mapbox=dict(style='carto-positron',
                              zoom=3, 
                              center = {"lat": 37.0902, "lon": -95.7129},
                              )) 
fig.update_layout(title_text="",
                  title_x=0.5,
                  margin={"r":10,"t":60,"l":0,"b":0})

button1 = dict(method= 'update',
               label='AAQ-fresh-surfacewater-withdrawal to SRD, mgd',
               args=[{"z": [df['AAQ-fresh-surfacewater-withdrawal to SRD, mgd']]}])

button2 = dict(method= 'update',
               label='EGD to ACI-pumping-fresh-surfacewater, bbtu',
               args=[{"z": [df['EGD to ACI-pumping-fresh-surfacewater, bbtu']]}])

fig.update_layout(updatemenus=[dict(active=0,
                                    buttons= [button1, button2],
                                   x=0,
                                    xanchor="left",
                                y=1.12,
                                yanchor="top")]
                                     )


fig.show()

## Code below is attempt to put all columns as buttons

In [ ]:
#hovertemplate = 'County: %{customdata[1]}'+'<br>State: %{customdata[2]}'+'<br>Value: %{z}<extra></extra>'

fig = go.Figure(go.Choroplethmapbox(geojson=counties, 
                                    locations=df['region'], 
                                    z=df['AAQ to SRD, mgd'],
                                    #featureidkey='properties.id',
                                    coloraxis="coloraxis",
                                    customdata=df,
                                    #hovertemplate= hovertemplate,
                                    marker_opacity=0.75, 
                                    marker_line_width=0.5))
fig.update_layout(coloraxis_colorscale='Viridis',
                  mapbox=dict(style='carto-positron',
                              zoom=3, 
                              center = {"lat": 37.0902, "lon": -95.7129},
                              )) 
fig.update_layout(title_text="Map of Select Flow Value",
                  title_x=0.5,
                  margin={"r":10,"t":60,"l":0,"b":0})

fig.update_layout(updatemenus=[dict(active=0,
                                    buttons= my_buttons,
                                   x=0.1,
                                    xanchor="left",
                                y=1.12,
                                yanchor="top")]
                                     )


fig.show()

In [ ]:
custom_data = df.copy()
custom_data

In [ ]:
# read in dataset
df = pd.read_csv(r"C:\Users\mong275\Local Files\Repos\full_run_output.csv", dtype={'region':str})
# add state and county columns
df['countyname'] = 'countyname'

# rearrange column order
col_list = df.columns[1:-1].to_list()
df = df[['region', 'countyname'] + col_list]

import json
json_file = r"C:\Users\mong275\Local Files\Repos\flow\flow\data\geojson-counties-fips.json"
f = open(json_file)
counties = json.load(f)


def plot_map(jsonfile:str, data=df, region_col=None, strip="total", center ={"lat": 37.0902, "lon": -95.7129}, ):

    # collect data
    df = data
    
    # collect geojson file
    f = open(jsonfile)
    geo_id = json.load(f)
    
    # create a flow name
    df['SOURCE'] = df['S1']
    df['TARGET'] = df['T1']

    # remove total from name
    remove = "-" + strip
    df['SOURCE'] = df['SOURCE'].str.replace(remove, "")
    df['TARGET'] = df['TARGET'].str.replace(remove, "")
    df['Link'] = df['SOURCE'] +  ' to ' + df['TARGET']+ ', ' +df['units']

    # pivot values
    df = pd.pivot_table(df, values='value', index=['region', 'countyname'],
                    columns=['Link'], aggfunc=np.sum)

    df = df.reset_index()                                       #reset index to remove multi-index from pivot table
    df = df.rename_axis(None, axis=1)                           #drop index name
    df.fillna(0,inplace=True)

    button_columns = region_col + 1
    # creates a button for each column with a value
    cols = df.columns[button_columns:].to_list()

    my_buttons = [dict(method= 'update',
        label=c,
        args=[{
        "z": [df[c]],
           "hovertemplate": "Region: %{customdata}"+\
           '<br>Value: %{z}<extra></extra>', # %<extra></extra> gets rid of trace box on right
        }]) for c in cols]
    
    starting_column = region_col + 1
    
    #stack region names
    mycustomdata = df['countyname'].to_list()
    
    # create figure
    fig = go.Figure(go.Choroplethmapbox(
                                    geojson = geo_id, 
                                    locations = df['region'], 
                                    z = df[df.columns[starting_column]],
                                    customdata = mycustomdata,
                                    hovertemplate = "Region: %{customdata}"+\
                                                    '<br>Value: %{z}<extra></extra>',
                                    coloraxis = "coloraxis",
                                    marker_opacity=0.75, 
                                    marker_line_width=0.1))
    
    #update map layout
    fig.update_layout(coloraxis_colorscale='viridis', 
                      mapbox=dict(style='carto-positron',
                                  zoom = 3, 
                                  center = center,
                              )) 
    # update title
    fig.update_layout(title_text="Map of Selected Flow Value in Each Region", title_x=0.5, margin={"r":10,"t":60,"l":0,"b":0})

    # update dropdown list
    fig.update_layout(updatemenus=[dict(active=0,
                                    buttons= my_buttons,
                                   x=0.1,
                                    xanchor="left",
                                y=1.12,
                                yanchor="top")]
                                     )


    fig.show()




In [ ]:
df.columns[1:]

In [ ]:
data_url = "https://raw.githubusercontent.com/empet/Datasets/master/Swiss-synthetic-data.csv"

df = pd.read_csv(data_url)
df.head()

mycustomdata = np.stack((df['canton-name'], df['2018'], df['2019']), axis=-1)
f = mycustomdata[0]
f

In [ ]:
mycustomdata = np.stack((df['region'], df[df.columns[region_col]]), axis=0)
mycustomdata[1][1]

In [ ]:
# TODO function below can be deleted
def prep_electricity_generation() -> pd.DataFrame:
    """ Provides a dataframe of electricity generation (MWh) and fuel use (BBTU) per year by generating technology type
    and by FIPS code. Can be used to estimate fuel use for electricity generation by type for each county
    and total electricity generation by county.

    :return:                DataFrame of fuel use in electricity generation and total generation by generation type
                            within each FIPS code
    """

    # read in electricity generation data by power plant id
    data = 'input_data\EIA923_Schedules_2_3_4_5_M_12_2015_Final_Revision.csv'
    df = pd.read_csv(data, skiprows=5)

    # read in power plant location data by power plant id
    df_gen_loc = prep_power_plant_location()
    df_loc = prep_water_use_2015()

    # read in power plant cooling type data
    cooling_data = r'input_data\2015_TE_Model_Estimates_USGS.csv'
    df_cooling = pd.read_csv(cooling_data,
                             usecols=['EIA_PLANT_ID', "COUNTY", 'STATE', 'NAME_OF_WATER_SOURCE', 'GENERATION_TYPE',
                                      'COOLING_TYPE', 'WATER_SOURCE_CODE', 'WATER_TYPE_CODE', 'WITHDRAWAL',
                                      'CONSUMPTION'])

    # remove unnecessary variables
    df_gen_loc = df_gen_loc[['FIPS', 'plant_code']]
    df = df[['Plant Id', "AER\nFuel Type Code", "Reported\nPrime Mover", "Total Fuel Consumption\nMMBtu",
             "Net Generation\n(Megawatthours)"]]

    # create a dictionary to bin power plant fuel types
    fuel_consumption_dict = {'SUN': 'solar',  # solar
                             'COL': 'coal',  # coal
                             'DFO': 'petroleum',  # distillate petroleum
                             "GEO": 'geothermal',  # geothermal
                             'HPS': 'hydro',  # hydro pumped storage
                             'HYC': 'hydro',  # hydro conventional
                             'MLG': 'biomass',  # biogenic municipal solid waste and landfill gas
                             'NG': 'natgas',  # natural gas
                             'NUC': 'nuclear',  # nuclear
                             'OOG': 'other',  # other gases
                             'ORW': 'other',  # other renewables
                             'OTH': 'other',  # other
                             'PC': 'petroleum',  # petroleum coke
                             'RFO': 'petroleum',  # residual petroleum
                             'WND': 'wind',  # wind
                             'WOC': 'coal',  # waste coal
                             'WOO': 'petroleum',  # waste oil
                             'WWW': 'biomass'}  # wood and wood waste

    prime_mover_dict = {'HY': 'instream',
                        'CA': 'combinedcycle',
                        'CT': 'combinedcycle',
                        'ST': 'steam',
                        'GT': 'combustionturbine',
                        'IC': 'internalcombustion',
                        'WT': 'onshore',
                        'PS': 'pumpedstorage',
                        'PV': 'photovoltaic',
                        'CS': 'combinedcycle',
                        'CE': 'compressedair',
                        'BT': 'binarycycle',
                        'OT': 'other',
                        'FC': 'fuelcell',
                        'CP': 'csp',
                        'BA': 'battery',
                        'FW': 'flywheel'
                        }

    water_source_dict = {'SW': 'surfacewater',  # river, canal, bay
                         'GW': 'groundwater',  # well, aquifer
                         'PD': 'wastewater',  # PD = plant discharge
                         "-nr-": "surfacewater",  # all blanks assumed to be surface water
                         "GW & PD": "groundwater",  # all GW+PD are assumed to be groundwater only
                         "GW & SW": 'surfacewater',  # all GW+SW combinations are assumed to be SW
                         "OT": "surfacewater"
                         }

    water_type_dict = {'FR': 'fresh',
                       'SA': 'saline',
                       'OT': 'fresh',  # all other source is assumed to be fresh water
                       "FR & BE": 'fresh',  # all combinations with fresh and BE are assumed to be fresh
                       "BE": "fresh",  # reclaimed wastewater
                       "BR": "saline",  # all brackish should be changed to saline
                       "": "fresh"}  # fill blanks with fresh water

    cooling_dict = {'COMPLEX': 'complex',
                    'ONCE-THROUGH FRESH': 'oncethrough',
                    'RECIRCULATING TOWER': 'tower',
                    'RECIRCULATING POND': 'pond',
                    'ONCE-THROUGH SALINE': 'oncethrough'}

    # rename columns in power plant generation data file
    df = df.rename(columns={"Plant Id": "plant_code"})
    df = df.rename(columns={"AER\nFuel Type Code": "fuel_type"})
    df = df.rename(columns={"Reported\nPrime Mover": "prime_mover"})
    df = df.rename(columns={"Total Fuel Consumption\nMMBtu": "fuel_amt"})
    df = df.rename(columns={"Net Generation\n(Megawatthours)": "generation_mwh"})

    # changing string columns to numeric
    string_col = df.columns[3:]  # create list of string columns
    for col in string_col:
        df[col] = df[col].str.replace(r"[^\w ]", '', regex=True)  # replace any non alphanumeric values
        df[col] = df[col].astype(float)  # convert to float

    # removing power plant generation rows that should not be included
    df = df[df.plant_code != 99999]  # removing state level estimated differences rows

    # changing from annual values to daily
    df['fuel_amt'] = df['fuel_amt'] / 365
    df['generation_mwh'] = df['generation_mwh'] / 365

    # dropping power plants with zero fuel use and zero output
    index_list = df[(df['fuel_amt'] <= 0) & (df['generation_mwh'] <= 0)].index  # list of indices with both zero values
    df.drop(index_list, inplace=True)  # dropping rows with zero fuel and zero generation amount

    # using fuel type dictionary to bin fuel types
    df['fuel_type'] = df['fuel_type'].map(fuel_consumption_dict)  # bin fuel types

    # using prime_mover_dict to bin prime mover types
    df['prime_mover'] = df['prime_mover'].map(prime_mover_dict)  # bin fuel types

    # converting units to billion btu from million btu
    df["fuel_amt"] = df["fuel_amt"] / 1000

    # grouping rows by both plant code and fuel type
    df = df.groupby(['plant_code', 'fuel_type', 'prime_mover'], as_index=False).sum()

    # merging power plant location data with power plant generation data
    df = pd.merge(df, df_gen_loc, how='left', on='plant_code')
    df_test_gen = df.copy()

    # ABOVE creates a dataframe of bbtu fuel and mwh generation by day by FIPS

    # COOLING WATER DATA

    # estimate discharge location from source information
    df_cooling['OCEAN_DISCHARGE_MGD'] = np.where(df_cooling['NAME_OF_WATER_SOURCE'].str.contains('Ocean', regex=False),
                                                 df_cooling['WITHDRAWAL'] - df_cooling['CONSUMPTION'],
                                                 0)
    # gulf of mexico
    df_cooling['OCEAN_DISCHARGE_MGD'] = np.where(df_cooling['NAME_OF_WATER_SOURCE'].str.contains('Gulf', regex=False),
                                                 df_cooling['WITHDRAWAL'] - df_cooling['CONSUMPTION'],
                                                 df_cooling['OCEAN_DISCHARGE_MGD'])

    # only bays with saline water are ocean discharge (some bays are on lakes (e.g. Green Bay))
    df_cooling['OCEAN_DISCHARGE_MGD'] = np.where(df_cooling['NAME_OF_WATER_SOURCE'].str.contains('Gulf', regex=False) &
                                                 df_cooling['WATER_TYPE_CODE'] == "SA",
                                                 df_cooling['WITHDRAWAL'] - df_cooling['CONSUMPTION'],
                                                 df_cooling['OCEAN_DISCHARGE_MGD'])
    # harbors
    df_cooling['OCEAN_DISCHARGE_MGD'] = np.where(
        df_cooling['NAME_OF_WATER_SOURCE'].str.contains('Harbor', regex=False) &
        df_cooling['WATER_TYPE_CODE'] == "SA",
        df_cooling['WITHDRAWAL'] - df_cooling['CONSUMPTION'],
        df_cooling['OCEAN_DISCHARGE_MGD'])
    # channels
    df_cooling['OCEAN_DISCHARGE_MGD'] = np.where(
        df_cooling['NAME_OF_WATER_SOURCE'].str.contains('Channel', regex=False) &
        df_cooling['WATER_TYPE_CODE'] == "SA",
        df_cooling['WITHDRAWAL'] - df_cooling['CONSUMPTION'],
        df_cooling['OCEAN_DISCHARGE_MGD'])
    # sounds
    df_cooling['OCEAN_DISCHARGE_MGD'] = np.where(df_cooling['NAME_OF_WATER_SOURCE'].str.contains('Sound', regex=False) &
                                                 df_cooling['WATER_TYPE_CODE'] == "SA",
                                                 df_cooling['WITHDRAWAL'] - df_cooling['CONSUMPTION'],
                                                 df_cooling['OCEAN_DISCHARGE_MGD'])

    # all remaining discharge is to surface water
    df_cooling['SURFACE_DISCHARGE_MGD'] = np.where(df_cooling['OCEAN_DISCHARGE_MGD'] == 0,
                                                   df_cooling['WITHDRAWAL'] - df_cooling['CONSUMPTION'],
                                                   0)
    # Fix water source and type data
    # all surface water without a type is assumed to be fresh
    df_cooling['WATER_TYPE_CODE'] = np.where((df_cooling["WATER_SOURCE_CODE"] == "SW")
                                             & (df_cooling["WATER_TYPE_CODE"] == "-nr-"),
                                             "FR", df_cooling['WATER_TYPE_CODE'])

    # all groundwater without a type is assumed to be fresh
    df_cooling['WATER_TYPE_CODE'] = np.where((df_cooling["WATER_SOURCE_CODE"] == "GW")
                                             & (df_cooling["WATER_TYPE_CODE"] == "-nr-"),
                                             "FR", df_cooling['WATER_TYPE_CODE'])

    # apply dictionaries
    df_cooling['COOLING_TYPE'] = df_cooling['COOLING_TYPE'].map(cooling_dict)  # rename cooling types
    df_cooling['WATER_SOURCE_CODE'] = df_cooling['WATER_SOURCE_CODE'].map(water_source_dict)  # rename water sources
    df_cooling['WATER_TYPE_CODE'] = df_cooling['WATER_TYPE_CODE'].map(water_type_dict)  # rename water types
    df_cooling['WATER_TYPE_CODE'].fillna('fresh', inplace=True)
    df_cooling['WATER_SOURCE_CODE'].fillna('surfacewater', inplace=True)

    df_cooling = df_cooling[['EIA_PLANT_ID', 'COOLING_TYPE', 'WATER_SOURCE_CODE', 'WATER_TYPE_CODE', 'WITHDRAWAL',
                             'CONSUMPTION', 'SURFACE_DISCHARGE_MGD', 'OCEAN_DISCHARGE_MGD']]
    df_cooling = df_cooling.rename(columns={'EIA_PLANT_ID': 'plant_code'})
    #

    df = pd.merge(df, df_cooling, how='left', on='plant_code')

    no_cool_list = ['hydro', 'wind', 'solar']

    for item in no_cool_list:
        df["COOLING_TYPE"] = np.where(df['fuel_type'] == item, np.nan, df["COOLING_TYPE"])
    #
    ## splitting out fuel data into a separate dataframe and pivoting to get fuel (bbtu) as columns by type
    df_fuel = df[
        ["FIPS", "fuel_amt", "fuel_type", 'prime_mover', "COOLING_TYPE"]].copy()  # create a copy of fuel type data
    df_fuel['COOLING_TYPE'].fillna('nocooling', inplace=True)

    df_fuel["fuel_name"] = 'EGS_' + df_fuel["fuel_type"] + '_' + df_fuel["prime_mover"] + '_' \
                           + df_fuel['COOLING_TYPE'] + '_total_bbtu_from_EPD_' + df_fuel["fuel_type"] \
                           + '_total_total_total_bbtu'

    # create a copy of the fuel dataframe to get direct supply from direct demand
    df_supply = df_fuel.copy()
    df_supply = df_supply[df_supply.fuel_type != 'coal']
    df_supply = df_supply[df_supply.fuel_type != 'petroleum']
    df_supply = df_supply[df_supply.fuel_type != 'biomass']
    df_supply = df_supply[df_supply.fuel_type != 'natgas']

    df_supply["supply_name"] = 'EPD_' + df_supply["fuel_type"] + '_total_total_total_bbtu_from_EPS_' + df_supply[
        "fuel_type"] + '_total_total_total_bbtu'
    df_supply = pd.pivot_table(df_supply, values='fuel_amt', index=['FIPS'], columns=['supply_name'],
                               aggfunc=np.sum)  # pivot
    df_supply = df_supply.reset_index()  # reset index to remove multi-index from pivot table
    df_supply = df_supply.rename_axis(None, axis=1)  # drop index name
    df_supply.fillna(0, inplace=True)  # fill nan with zero

    # example: EGS_coal_stream_oncethrough_total_bbtu_from_EPD_coal_total_total_total_bbtu

    df_fuel = pd.pivot_table(df_fuel, values='fuel_amt', index=['FIPS'], columns=['fuel_name'], aggfunc=np.sum)  # pivot
    df_fuel = df_fuel.reset_index()  # reset index to remove multi-index from pivot table
    df_fuel = df_fuel.rename_axis(None, axis=1)  # drop index name
    df_fuel.fillna(0, inplace=True)  # fill nan with zero
    #
    ## splitting out generation data into a separate dataframe and pivoting to get generation (mwh) as columns by type
    df_gen = df[["FIPS", "generation_mwh", 'fuel_amt', 'prime_mover', "fuel_type",
                 'COOLING_TYPE']].copy()  # create a copy of generation data
    df_gen['COOLING_TYPE'].fillna('nocooling', inplace=True)
    df_gen['generation_mwh'] = df_gen['generation_mwh'].apply(convert_mwh_bbtu)  # convert to bbtu from mwh

    # calculate rejected energy fractions
    df_gen['rej_fraction'] = np.where(df_gen['fuel_amt'] > 0, df_gen['generation_mwh'] / df_gen['fuel_amt'], 0)

    df_demand = df_gen.copy()
    df_demand['demand_fraction'] = 1 - df_demand['rej_fraction']

    df_gen["fuel_type_name"] = 'EGS_' + df_gen["fuel_type"] + '_' + df_gen["prime_mover"] + '_' \
                               + df_gen[
                                   'COOLING_TYPE'] + "_total_bbtu_to_REJ_total_total_total_total_bbtu_fraction"  # add naming
    #
    df_gen = pd.pivot_table(df_gen, values='rej_fraction', index=['FIPS'], columns=['fuel_type_name'], aggfunc=np.sum)
    df_gen = df_gen.reset_index()  # reset index to remove multi-index from pivot table
    df_gen = df_gen.rename_axis(None, axis=1)  # drop index name
    df_gen.fillna(0, inplace=True)  # fill nan with zero

    # create electricity demand discharge fraction
    df_demand["fuel_type_name"] = 'EGS_' + df_demand["fuel_type"] + '_' + df_demand["prime_mover"] + '_' \
                                  + df_demand[
                                      'COOLING_TYPE'] + "_total_bbtu_to_EGD_total_total_total_total_bbtu_fraction"  # add naming
    df_demand = pd.pivot_table(df_demand, values='demand_fraction', index=['FIPS'], columns=['fuel_type_name'],
                               aggfunc=np.sum)
    df_demand = df_demand.reset_index()  # reset index to remove multi-index from pivot table
    df_demand = df_demand.rename_axis(None, axis=1)  # drop index name
    df_demand.fillna(0, inplace=True)  # fill nan with zero

    # create water intensity values

    # create water withdrawal source fractions
    df_cooling_int = df[["FIPS", 'plant_code', 'prime_mover', "generation_mwh", "fuel_type", 'COOLING_TYPE',
                         'WATER_TYPE_CODE', 'WATER_SOURCE_CODE', 'WITHDRAWAL']].copy()

    df_cooling_int["water_intensity_name"] = 'EGS_' + df['fuel_type'] + '_' + df['prime_mover'] + '_' \
                                             + df['COOLING_TYPE'] + '_total_mgd_from_EGS_' \
                                             + df['fuel_type'] + '_' + df['prime_mover'] + '_' \
                                             + df['COOLING_TYPE'] + '_total_bbtu_intensity'

    cooling_only = df_cooling_int[df_cooling_int.COOLING_TYPE != 'nocooling'].groupby('plant_code',
                                                                                      as_index=False).count()
    cooling_only = cooling_only.rename(columns={'FIPS': 'count'})
    cooling_only = cooling_only[['plant_code', 'count']]
    df_cooling_int = pd.merge(df_cooling_int, cooling_only, how='left', on='plant_code')
    df_cooling_int['count'].fillna(1, inplace=True)
    df_cooling_int = df_cooling_int.dropna(subset=["WITHDRAWAL"])
    df_cooling_int['WITHDRAWAL'] = df_cooling_int['WITHDRAWAL'] / df_cooling_int['count']

    df_cooling_int['generation_mwh'] = df_cooling_int['generation_mwh'].apply(
        convert_mwh_bbtu)  # convert to bbtu from mwh
    df_cooling_int['intensity'] = np.where(df_cooling_int['generation_mwh'] > 0,
                                           df_cooling_int['WITHDRAWAL'] / df_cooling_int['generation_mwh'],
                                           0)

    df_cooling_int = pd.pivot_table(df_cooling_int, values='intensity', index=['FIPS'],
                                    columns=['water_intensity_name'], aggfunc=np.mean)
    df_cooling_int = df_cooling_int.reset_index()  # reset index to remove multi-index from pivot table
    df_cooling_int = df_cooling_int.rename_axis(None, axis=1)  # drop index name
    df_cooling_int.fillna(0, inplace=True)  # fill nan with zero

    # create water withdrawal source fractions
    df_cooling_w = df[["FIPS", 'plant_code', 'prime_mover', "fuel_type", 'COOLING_TYPE',
                       'WATER_TYPE_CODE', 'WATER_SOURCE_CODE', 'WITHDRAWAL']].copy()

    df_cooling_w["water_withdrawal_name"] = 'EGS_' + df['fuel_type'] + '_' + df['prime_mover'] + '_' \
                                            + df['COOLING_TYPE'] + '_total_mgd_from_WSW_' \
                                            + df_cooling_w["WATER_TYPE_CODE"] + '_' + df_cooling_w["WATER_SOURCE_CODE"] \
                                            + '_total_total_mgd_fraction'

    cooling_only = df_cooling_w[df_cooling_w.COOLING_TYPE != 'nocooling'].groupby('plant_code', as_index=False).count()
    cooling_only = cooling_only.rename(columns={'FIPS': 'count'})
    cooling_only = cooling_only[['plant_code', 'count']]
    df_cooling_w = pd.merge(df_cooling_w, cooling_only, how='left', on='plant_code')
    df_cooling_w['count'].fillna(1, inplace=True)
    df_cooling_w = df_cooling_w.dropna(subset=["WITHDRAWAL"])
    df_cooling_w['WITHDRAWAL'] = df_cooling_w['WITHDRAWAL'] / df_cooling_w['count']

    # convert withdrawal to value of 1 to be 100% fraction of source
    df_cooling_w['WITHDRAWAL'] = np.where(df_cooling_w['WITHDRAWAL'] > 0, 1, 0)

    df_cooling_w = pd.pivot_table(df_cooling_w, values='WITHDRAWAL', index=['FIPS'],
                                  columns=['water_withdrawal_name'], aggfunc=np.mean)
    df_cooling_w = df_cooling_w.reset_index()  # reset index to remove multi-index from pivot table
    df_cooling_w = df_cooling_w.rename_axis(None, axis=1)  # drop index name
    df_cooling_w.fillna(0, inplace=True)  # fill nan with zero

    # create consumption fractions
    df_cooling_c = df[
        ["FIPS", 'plant_code', 'prime_mover', "fuel_type", 'COOLING_TYPE', 'WATER_TYPE_CODE', 'WATER_SOURCE_CODE',
         'CONSUMPTION', 'WITHDRAWAL']].copy()
    df_cooling_c["water_consumption_name"] = 'EGS_' + df['fuel_type'] + '_' + df['prime_mover'] + '_' \
                                             + df[
                                                 'COOLING_TYPE'] + '_total_mgd_to_CMP_total_total_total_total_mgd_fraction'

    df_cooling_c = pd.merge(df_cooling_c, cooling_only, how='left', on='plant_code')
    df_cooling_c['count'].fillna(1, inplace=True)
    df_cooling_c = df_cooling_c.dropna(subset=["CONSUMPTION"])
    df_cooling_c['CONSUMPTION'] = df_cooling_c['CONSUMPTION'] / df_cooling_c['count']

    df_cooling_c['cons_fraction'] = df_cooling_c['CONSUMPTION'] / df_cooling_c['WITHDRAWAL']

    df_cooling_c = pd.pivot_table(df_cooling_c, values='cons_fraction', index=['FIPS'],
                                  columns=['water_consumption_name'],
                                  aggfunc=np.mean)
    df_cooling_c = df_cooling_c.reset_index()  # reset index to remove multi-index from pivot table
    df_cooling_c = df_cooling_c.rename_axis(None, axis=1)  # drop index name
    df_cooling_c.fillna(0, inplace=True)  # fill nan with zero

    # surface discharge fraction
    df_cooling_sd = df[
        ["FIPS", 'plant_code', 'prime_mover', "fuel_type", 'COOLING_TYPE', 'WATER_TYPE_CODE', 'WATER_SOURCE_CODE',
         'SURFACE_DISCHARGE_MGD', 'WITHDRAWAL']].copy()
    df_cooling_sd["sd_name"] = 'EGS_' + df['fuel_type'] + '_' + df['prime_mover'] + '_' \
                               + df['COOLING_TYPE'] + '_total_mgd_to_SRD_total_total_total_total_mgd_fraction'

    df_cooling_sd = pd.merge(df_cooling_sd, cooling_only, how='left', on='plant_code')
    df_cooling_sd['count'].fillna(1, inplace=True)
    df_cooling_sd = df_cooling_sd.dropna(subset=["SURFACE_DISCHARGE_MGD"])
    df_cooling_sd['SURFACE_DISCHARGE_MGD'] = df_cooling_sd['SURFACE_DISCHARGE_MGD'] / df_cooling_sd['count']

    df_cooling_sd['sd_fraction'] = df_cooling_sd['SURFACE_DISCHARGE_MGD'] / df_cooling_sd['WITHDRAWAL']

    df_cooling_sd = pd.pivot_table(df_cooling_sd, values='sd_fraction', index=['FIPS'], columns=['sd_name'],
                                   aggfunc=np.sum)
    df_cooling_sd = df_cooling_sd.reset_index()  # reset index to remove multi-index from pivot table
    df_cooling_sd = df_cooling_sd.rename_axis(None, axis=1)  # drop index name
    df_cooling_sd.fillna(0, inplace=True)  # fill nan with zero

    # ocean discharge fractions
    df_cooling_od = df[
        ["FIPS", 'plant_code', 'prime_mover', "fuel_type", 'COOLING_TYPE', 'WATER_TYPE_CODE', 'WATER_SOURCE_CODE',
         'OCEAN_DISCHARGE_MGD', 'WITHDRAWAL']].copy()
    df_cooling_od["od_name"] = 'EGS_' + df['fuel_type'] + '_' + df['prime_mover'] + '_' \
                               + df['COOLING_TYPE'] + '_total_mgd_to_OCD_total_total_total_total_mgd_fraction'

    df_cooling_od = pd.merge(df_cooling_od, cooling_only, how='left', on='plant_code')
    df_cooling_od['count'].fillna(1, inplace=True)
    df_cooling_od = df_cooling_od.dropna(subset=["OCEAN_DISCHARGE_MGD"])
    df_cooling_od['OCEAN_DISCHARGE_MGD'] = df_cooling_od['OCEAN_DISCHARGE_MGD'] / df_cooling_od['count']

    df_cooling_od['od_fraction'] = df_cooling_od['OCEAN_DISCHARGE_MGD'] / df_cooling_od['WITHDRAWAL']

    df_cooling_od = pd.pivot_table(df_cooling_od, values='OCEAN_DISCHARGE_MGD', index=['FIPS'], columns=['od_name'],
                                   aggfunc=np.sum)
    df_cooling_od = df_cooling_od.reset_index()  # reset index to remove multi-index from pivot table
    df_cooling_od = df_cooling_od.rename_axis(None, axis=1)  # drop index name
    df_cooling_od.fillna(0, inplace=True)  # fill nan with zero

    # merge dataframes
    df_fuel = pd.merge(df_loc, df_fuel, how='left', on='FIPS').fillna(0)
    # df_supply = pd.merge(df_loc, df_supply, how='left', on='FIPS').fillna(0)
    df_gen = pd.merge(df_loc, df_gen, how='left', on='FIPS').fillna(0)
    df_demand = pd.merge(df_loc, df_demand, how='left', on='FIPS').fillna(0)
    df_cooling_int = pd.merge(df_loc, df_cooling_int, how='left', on='FIPS').fillna(0)
    df_cooling_w = pd.merge(df_loc, df_cooling_w, how='left', on='FIPS').fillna(0)
    df_cooling_c = pd.merge(df_loc, df_cooling_c, how='left', on='FIPS').fillna(0)
    df_cooling_sd = pd.merge(df_loc, df_cooling_sd, how='left', on='FIPS').fillna(0)
    df_cooling_od = pd.merge(df_loc, df_cooling_od, how='left', on='FIPS').fillna(0)

    # rem_list = [df_cooling_w, df_cooling_c, df_cooling_sd, df_cooling_od]
    out_df = pd.merge(df_fuel, df_gen, how='left', on=['FIPS', 'State', 'County'])
    out_df = pd.merge(out_df, df_demand, how='left', on=['FIPS', 'State', 'County'])
    # out_df = pd.merge(out_df, df_supply, how='left', on=['FIPS', 'State', 'County'])
    out_df = pd.merge(out_df, df_cooling_int, how='left', on=['FIPS', 'State', 'County'])
    out_df = pd.merge(out_df, df_cooling_w, how='left', on=['FIPS', 'State', 'County'])
    out_df = pd.merge(out_df, df_cooling_c, how='left', on=['FIPS', 'State', 'County'])
    out_df = pd.merge(out_df, df_cooling_sd, how='left', on=['FIPS', 'State', 'County'])
    out_df = pd.merge(out_df, df_cooling_od, how='left', on=['FIPS', 'State', 'County'])

    return df_test_gen

In [ ]:
df2 = df.groupby(['S1','t1','units'], as_index=False).sum()
df2

In [ ]:
df = flow.prep_water_use_2015()
df

In [ ]:
df.to_csv(r"C:\Users\mong275\Local Files\Repos\full_run_output.csv", index=False)

In [ ]:
df.to_csv(r"C:\Users\mong275\Local Files\Repos\flow\sample_data\full_output.csv")

In [ ]:
df2 = df.copy()
df2 = df2[df.region == '41071']
df2 = df2[df2.t1 == 'WWD']
df2 = df2[df2.units == 'bbtu']
df2.sort_values(by=['value'], inplace=True, ascending=False)

df2.replace('total', "", inplace=True)
df2['source_name'] = df2['S1'] +" " + df2['S2'] +" " + df2['S3'] +" " +df2['S4'] +" " +df2['S5']
df2['source_name'] = df2['source_name'].str.strip()
df2['target_name'] = df2['t1'] +" " + df2['t2'] +" " + df2['t3'] +" " +df2['t4'] +" " +df2['t5']
df2['target_name'] = df2['target_name'].str.strip()
df2['flow_name'] = df2['source_name'] + ' to ' + df2['target_name']
df2['flow_name'] = df2['flow_name'].str.strip()

flow_list = df2['flow_name'].head().to_list()
value_list = df2['value'].round(4).head().to_list()

dictionary = dict(zip(flow_list, value_list))


#print(f'The top flows to {sector_name} in {unit_type} in {region_name} are: ')
print(f'The top bbtu flows to WWD in 41071 are: ')
for name in dictionary:
    print(f'{name}        {dictionary[name]}  bbtu')
        

In [ ]:
d = flow.calculate()
d

In [ ]:
d['1_PWS_fresh_surfacewater_total_total_mgd']

In [ ]:
d = pd.read_csv(r"C:\Users\mong275\Local Files\Repos\flow\flow\data\configuration_data\test_data_01-28.csv")
d
n = flow.construct_nested_dictionary(d)
n

In [ ]:
d = flow.read_config('baseline', 'file1')
d

In [ ]:
with open(r'C:\Users\mong275\Local Files\Repos\flow\flow\data\configuration_data\config.yml') as file:
    dict_name = yaml.safe_load(file)
dict_name['collect_water']['file2']['path']

In [ ]:
with open('flow/configuration.yml', 'r') as file:
    prime_service = yaml.safe_load(file)
prime_service['rest']['url']

In [ ]:
b = flow.calc_energy_direct_demand_water_use()
b
for columns in b.columns:
    print (columns)
b

In [ ]:
split_types = flow.test_EP_flows()
split_types

In [ ]:
target_types = flow.test_water_sector_energy_discharge()
split_dict = flow.construct_nested_dictionary(target_types)

for x1 in split_dict:
    l1_energy_name = f'{x1}_bbtu'
    for x2 in split_dict[x1]:
        l2_energy_name = f'{x1}_{x2}_bbtu'
        for x3 in split_dict[x1][x2]:
            l3_energy_name = f'{x1}_{x2}_{x3}_bbtu'
            for x4 in split_dict[x1][x2][x3]:
                l4_energy_name = f'{x1}_{x2}_{x3}_{x4}_bbtu'
                for x5 in split_dict[x1][x2][x3][x4]:
                    l5_energy_name = f'{x1}_{x2}_{x3}_{x4}_{x5}_mgd'
                    if l5_energy_name in l5_dict:
                        for m_type in split_dict[x1][x2][x3][x4][x5]:
                            print(split_dict[x1][x2][x3][x4][x5][m_type])

In [ ]:

p = flow.test_ww_param()
#f = flow.test_EP_flows()

d = flow.calc_collect_water_use()
for col in d.columns:
    print(col)

In [ ]:
b = flow.test_water_sector_param()
d = flow.construct_nested_dictionary(b)


for x1 in d:
    l1_demand_name = f'{x1}_mgd'
    l1_demand_value = 0
    for x2 in d[x1]:
        l2_demand_name = f'{x1}_{x2}_mgd'
        l2_demand_value = 0
        for x3 in d[x1][x2]:
            l3_demand_name = f'{x1}_{x2}_{x3}_mgd'
            l3_demand_value = 0
            for x4 in d[x1][x2][x3]:
                l4_demand_name = f'{x1}_{x2}_{x3}_{x4}_mgd'
                l4_demand_value = 0
                for x5 in d[x1][x2][x3][x4]:
                    l5_demand_name = f'{x1}_{x2}_{x3}_{x4}_{x5}_mgd'
                    l5_demand_value = 0
                    for s1 in d[x1][x2][x3][x4][x5]:
                        for s2 in d[x1][x2][x3][x4][x5][s1]:
                            l1_supply_name = f'{s1}_to_{x1}_mgd'

                            for s3 in d[x1][x2][x3][x4][x5][s1][s2]:
                                l2_supply_name = f'{s1}_{s2}_to_{x1}_{x2}_mgd'
                                for s4 in d[x1][x2][x3][x4][x5][s1][s2][s3]:
                                    l3_supply_name = f'{s1}_{s2}_{s3}_to_{x1}_{x2}_{x3}_mgd'
                                    l5_demand_value = 0
                                    for s5 in d[x1][x2][x3][x4][x5][s1][s2][s3][s4]:
                                        l4_supply_name = f'{s1}_{s2}_{s3}_{s4}_to_{x1}_{x2}_{x3}_{x4}_mgd'
                                        l5_supply_name = f'{s1}_{s2}_{s3}_{s4}_{s5}_to_{x1}_{x2}_{x3}_{x4}_{x5}_mgd'
                                        print(l5_supply_name)

In [ ]:

#pd.set_option('display.max_rows', None)
data = flow.configure_data()
source_parameters = flow.get_water_demand_source_parameters()
target_parameters = flow.get_water_demand_target_parameters()
x = flow.calc_water_to_nonwater_sectors(data=data, source_parameters=source_parameters, target_parameters=target_parameters, output='l5')
x.sum()

In [ ]:
d = flow.calc_pws_discharge()
d.sum()

In [ ]:
d = flow.get_water_use_2015()
d['PS-WSWSa'].sum()

In [ ]:
df = flow.prep_electricity_demand_data()
demand_columns = df.columns[1:5].to_list()
demand_columns
for d in demand_columns:
    df[d] = df[d] * df['pop_weight']
    df[d] = df[d].round(4)
df.sum()

In [ ]:
d = flow.prep_electricity_demand_data()
d.dtypes

In [ ]:
df = flow.get_state_electricity_demand_data()

    # build renaming dictionary
rename_dict = {"RESIDENTIAL": "EG_total_total_total_total_to_RES_total_total_total_total_bbtu",
                   "COMMERCIAL": "EG_total_total_total_total_to_CO_total_total_total_total_bbtu",
                   "INDUSTRIAL": "EG_total_total_total_total_to_IN_total_total_total_total_bbtu",
                   "TRANSPORTATION": "EG_total_total_total_total_to_TR_total_total_total_total_bbtu"}

    # prep dataframe
df = df.dropna(subset=["Month"])  # drop rows where month is blank
df = df.dropna(subset=["Ownership"])  # Drop state totals and state adjustments
df = df[df.Ownership != "Behind the Meter"]  # removing behind the meter generation
df = df.groupby("State", as_index=False).sum()  # get total by state
df = df[["State", "RESIDENTIAL", "COMMERCIAL", "INDUSTRIAL", "TRANSPORTATION"]]

    # convert electricity demand values from mwh to bbtu
column_list = df.columns[1:]
for col in column_list:
    df[col] = df[col].apply(flow.convert_mwh_bbtu)

    # rename columns to add descriptive language
df.rename(columns=rename_dict, inplace=True)
    

    # split out into county values based on percent of state population
df = flow.calc_population_county_weight(df)
demand_columns = df.columns[1:5].to_list()
for d in demand_columns:
    df[d] = df[d] * df['pop_weight']
    df[d] = df[d].round(4)
df

In [ ]:
y = flow.construct_nested_dictionary(flow.get_fuel_demand_target_parameters())
#len(flow.get_fuel_demand_source_parameters())
#x = flow.get_fuel_demand_source_parameters()
y

In [ ]:
x = flow.prep_fuel_demand_data()
x

In [ ]:
l1_df = pd.DataFrame.from_dict(x, orient='index').transpose()


In [ ]:

df = pd.DataFrame()
for i in range(len(data)):
    df = df.append({'A': 0}, ignore_index=True)
df

In [ ]:
for col in x.columns:
    print(col)

In [ ]:
df.sum()

In [ ]:
fail_parameter_data = flow.get_electricity_generation_efficiency_parameters()
fail_parameter_data = fail_parameter_data.drop(fail_parameter_data.columns[2], axis=1)
fail_parameter_data

In [ ]:
parameters = flow.get_electricity_generation_efficiency_parameters()
test_dict = flow.construct_nested_dictionary(parameters)
test_fuel_type = list(test_dict.keys())[0]
test_sub_fuel_type = list(test_dict[test_fuel_type].keys())[0]

test_sub_fuel_type

In [ ]:
#df = flow.calc_electricity_energy_discharge(data=d, total=True)
df['eg_generation_biomass_to_es_bbtu'][4]

In [ ]:
df.sum()

In [ ]:
df = flow.get_electricity_generation_efficiency_parameters()
x = flow.construct_nested_dictionary(df)

x['biomass']['total']['efficiency']

In [ ]:
efficiency_dict = {'biomass':{'total'}, 'coal':{'total'}, 'geothermal':{'total'}, 'hydro':{'total'}, 'natgas':{'total'}, 'nuclear':{'total'},
                       'oil':{'total'}, 'other':{'total'}, 'solar':{'total'}, 'wind':{'total'}}
efficiency_dict

In [ ]:
d = df.groupby(group1).apply(lambda x: dict(zip(x[parameter])))

In [ ]:
group1 = df.columns[0]
group2 = df.columns[1]
group3 = df.columns[2]
group4 = df.columns[3]
parameter = df.columns[-2]
value = df.columns[-1]


d = df.groupby(group1).apply(lambda x: dict(zip(x[parameter], x[value])))
d = d.to_dict()
d


In [ ]:
d = df.groupby(group1).apply(lambda a: dict(a.groupby(group2).apply(lambda b: dict(b.groupby(group3).apply(
            lambda b: dict(b.groupby(group4).apply(lambda x: dict(zip(x[parameter], x[value])))))))))

In [ ]:
main_grouper = df.columns[0]
nest_list = df.columns[1:].to_list()
second_list =  df.columns[2:]
sub_nest_grouper = nest_list[0]

In [ ]:
fuel_dict = df.groupby(main_grouper)[nest_list].apply(lambda x: x.set_index([nest_list]).to_dict(orient='index')).to_dict()
fuel_dict

In [ ]:
for item in res:
    print(res['biomass'])
    

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_list = data.select_dtypes(include=numerics).columns.tolist()
record_list = []
for column in numeric_list:
    x = data.columns.get_loc(column)
    record_list.append(x)
length = min(record_list)


main_grouper = data.columns[0]
nest_list = data.columns[1:].to_list()
sub_nest_grouper = nest_list[0]
fuel_dict = data.groupby(main_grouper)[nest_list].apply(lambda x: x.set_index(sub_nest_grouper).to_dict(orient='index')).to_dict()
fuel_dict

In [ ]:
main_grouper = data.columns[1]
nest_list = data.columns[1:].to_list()
sub_nest_grouper = nest_list[0]

    # no sub-fuel types
main_type_unique = data[main_grouper].is_unique
sub_type_unique = data[sub_nest_grouper].is_unique

if main_type_unique and sub_type_unique:
    fuel_dict = data.set_index(main_grouper).to_dict(orient='index')
elif not main_type_unique and not sub_type_unique:

    fuel_dict = data.groupby(main_grouper)[nest_list].apply(lambda x: x.set_index(sub_nest_grouper).to_dict(orient='index')).to_dict()
else:
    pass

fuel_dict

In [ ]:
#data.apply(lambda x: x.set_index('fuel_types').to_dict(orient='index'))
nest_list = data.columns[2:].to_list()
fuel_dict = data.groupby('fuel_type')#[nest_list].apply(lambda x: x.set_index('fuel_type').to_dict(orient='index')).to_dict()
fuel_dict = data.set_index('fuel_type').to_dict(orient='index')
fuel_dict

In [ ]:
data.head()
d = flow.construct_dictionary(data)
d

In [ ]:
df.columns[1]
nest_list = df.columns[2:].to_list()
nest_list[0]

In [ ]:
my_dict = {1:{'name':'x', 'age':24}, 2:{'name':'y', 'age':21}, 3:{'name':'z', 'age':25}}

#my_dict
#def count(d):
#    return max(count(v) if isinstance(v,dict) else 0 for v in d.values()) + 1

#print(count(my_dict))

In [ ]:
    nested_dict = df.groupby(main_grouper)[nest_list].apply(lambda x: x.set_index(sub_nest_grouper).to_dict(orient='index')).to_dict()


In [ ]:
import flow

In [ ]:
x = fresh_groundwater_coal_generation_mgd + fresh_surfacewater_coal_generation_mgd + fresh_wastewater_coal_generation_mgd + saline_groundwater_coal_generation_mgd + saline_surfacewater_coal_generation_mgd
x
z = coal_generation_surface_discharge_mgd + coal_generation_ocean_discharge_mgd
x - coal_generation_consumption_mgd 

In [ ]:
z

In [ ]:
df = df.groupby('fuel_type')[['sub_fuel','fuel_efficiency','parameter2']].apply(lambda x: x.set_index('sub_fuel').to_dict(orient='index')).to_dict()
df

In [ ]:
round(df).sum()

In [ ]:
54+45

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
48384*0.00016149393844372542

In [ ]:
df.sum()

In [ ]:

df['total_water'] = df['fresh_surfacewater_natgas_unconventional_mgd'] + df['fresh_groundwater_natgas_unconventional_mgd']
df['intensity'] = np.where(df['natgas_unconventional_production_bbtu']>0, df['total_water']/(df['natgas_unconventional_production_bbtu']/365), 0)
df['sw_pct'] = np.where( df['total_water']>0, df['fresh_surfacewater_natgas_unconventional_mgd']/df['total_water'], 0)
df['gw_pct'] = np.where( df['total_water']>0, df['fresh_groundwater_natgas_unconventional_mgd']/df['total_water'], 0)

df.groupby("State", as_index=False).sum()
x = df['intensity'].mean()
x

In [ ]:

# initialize data of lists.
d_list = {'Name':['Tom', 'Tom','nick', 'krish', 'jack'],
        'Age':[20, 22, 21, 19, 18],
         'height':[20,21, 21, 19, 18]}
 
# Create DataFrame
d_list = pd.DataFrame(d_list)
d_list.head()

d_list = d_list.groupby('Name').apply(lambda x: x.set_index('Age').to_dict(orient='index')).to_dict()
d_list

In [ ]:
df3 = pd.DataFrame([
           ['A', 123, 1], 
           ['B', 345, 5], 
           ['C', 712, 4],
           ['B', 768, 2], 
           ['B', 768, 3], 
           ['A', 123, 9], 
           ['C', 178, 6], 
           ['C', 178, 5],  
           ['A', 321, 3]])

d = {k: f.groupby('subgroup')['selectedCol'].apply(list).to_dict()
     for k, f in df.groupby('maingroup')}
d

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
df = data.copy()

In [ ]:
# def calc_coal_production_water_use(data: pd.DataFrame, mine_types = None, water_types=None, discharge_types = None,
#                                   regions=3, total=False):
#    """calculates total water use in hydroelectric generation by region.
#
#        Water use is determined by applying hydropower water intensity rates (mg/bbtu) to daily hydropower generation
#        (bbtu) to get mgd in each region. Water use can be interpreted as the amount of water that passes through
#        the hydropower facility on a daily basis, based on average power generation per day. If region-level hydropower
#        water intensity rates are not available, an intensity assumption is applied to all hydropower generation.
#        Surface discharge from hydropower is also calculated. This is assumed to be equal to water use in hydropower.
#        No water is assumed to be consumed or evaporated given that the water included in this calculation only
#        includes water that instantaneously passes through the hydropower facility and is immediately discharged back
#        to the surface water source.
#
#        :param data:                        DataFrame of input data
#        :type data:                         DataFrame
#
#        :param hydro_water_intensity:       The assumed million gallons required to get a billion btu from hydropower
#        :type hydro_water_intensity:        flt
#
#        :param regions:                     gives the number of columns in the dataset that should be treated as region
#                                            identifiers (e.g. "Country", "State"). Reads from the first column in the
#                                            dataframe onwards.
#        :type regions:                      int
#
#        :return:                            DataFrame of total water use by hydropower generation (mgd) and hydropower
#                                            discharge to surface waters (mgd) by region.
#
#        """
#
#    if mine_types is None:
#        mine_type_dict = {'surface': 0.000007, 'ground': 0.000029}
#    else:
#        mine_type_dict = mine_types
#
#    if water_types is None:
#        water_type_dict = {'fresh': {'surfacewater': {'flow_fraction': .5, 'consumption_fraction': .5},
#                                    'groundwater': {'flow_fraction': .5, 'consumption_fraction': .5}},
#                           'saline': {'surfacewater': {'flow_fraction': .5, 'consumption_fraction': .5},
#                                    'groundwater': {'flow_fraction': .5, 'consumption_fraction': .5}}}
#    else:
#        water_type_dict = water_types
#
#    if discharge_types is None:
#        discharge_type_dict = {'surface': 1}
#    else:
#        discharge_type_dict = discharge_types
#
#    # load data
#    df = data
#
#    region_list = df.columns[:regions].tolist()
#    output_df = df[region_list].copy()
#    total_df = df[region_list].copy()
#
#    for water_type in water_type_dict:
#        for water_source in water_type_dict[water_type]:
#            coal_water_total_name = f'{water_type}_{water_source}_coal_mgd'
#            output_df[coal_water_total_name] = 0
#            total_df[coal_water_total_name] = 0
#
#    # calculate total water to all mining (inclusive of coal)
#    df['total_water_mining_mgd'] = 0
#    for water_type in water_type_dict:
#        for water_source in water_type_dict[water_type]:
#            mining_water_flow_name = f'{water_type}_{water_source}_mining_mgd'
#            if mining_water_flow_name in df.columns:
#                df['total_water_mining_mgd'] = df['total_water_mining_mgd'] + df[mining_water_flow_name]
#            else:
#                pass
#
#    # calculate percent of total mining flows by water type and water source
#    for water_type in water_type_dict:
#        for water_source in water_type_dict[water_type]:
#
#            mining_water_flow_pct_name = f'{water_type}_{water_source}_mining_pct'
#
#            if mining_water_flow_pct_name in df.columns: #if the percent of water flows to coal mining in data
#                pass
#
#            else:  # if only water flows to all mining in data
#                mining_water_flow_name = f'{water_type}_{water_source}_mining_mgd'
#                if mining_water_flow_name in df.columns:  # if water type and water source to all mining in data
#                    df[mining_water_flow_pct_name] = np.where(df['total_water_mining_mgd'] > 0,
#                                                          (df[mining_water_flow_name] / df['total_water_mining_mgd']),
#                                                          0)
#                else:  # if no breakdown of water source and water type to all mining in data
#                    df[mining_water_flow_pct_name] = water_type_dict[water_type][water_source]['flow_fraction']
#
#            # calculate water in coal mining for each mine type by water type and water source
#            for mine_type in mine_type_dict:
#                coal_shortton_name = f'{mine_type}_coal_shortton'
#                if coal_shortton_name in df.columns:
#                    coal_water_intensity_name = f'{mine_type}_coal_water_intensity_mg_per_shortton'
#                    coal_water_total_name = f'{water_type}_{water_source}_coal_mgd'
#                    if coal_water_intensity_name in df.columns:  # if regional water intensity values available
#                        coal_water_mgd_name = f"{water_type}_{water_source}_{mine_type}_coal_mgd"
#                        output_df[coal_water_mgd_name] = df[coal_water_intensity_name] \
#                                                         * (df[coal_shortton_name]/365) \
#                                                         * df[mining_water_flow_pct_name]
#                        output_df[coal_water_total_name] = output_df[coal_water_total_name] \
#                                                           + output_df[coal_water_mgd_name]
#                        total_df[coal_water_total_name] = total_df[coal_water_total_name] \
#                                                          + output_df[coal_water_mgd_name]
#
#                    else:  # if no regional water intensity available, use assumption
#                        coal_water_mgd_name = f"{water_type}_{water_source}_{mine_type}_coal_mgd"
#                        output_df[coal_water_mgd_name] = mine_type_dict[mine_type] \
#                                                         * (df[coal_shortton_name]/365) \
#                                                         * df[mining_water_flow_pct_name]
#                        output_df[coal_water_total_name] = output_df[coal_water_total_name] \
#                                                           + output_df[coal_water_mgd_name]
#                        total_df[coal_water_total_name] = total_df[coal_water_total_name] \
#                                                          + output_df[coal_water_mgd_name]
#                else:  # if no coal mining available
#                    pass
#
## TODO pick things up here
#
#            # calculate consumption of water in coal mining by water type and water source
#
#            # look for region- mining consumption fraction, apply it
#
#            # otherwise use dictionary values water_type_dict[water_type][water_source]['consumption_fraction']
#
#            ## look for region- mining discharge fraction, apply it
#            # for discharge_type in discharge_type dictionary:
#                #discharge_type_name = f'{water_type}_{water_source}_{mine_type}_{discharge_type}_discharge_mgd
#                #discharge_pct_name = f'{water_type}_{water_source}_{mine_type}_{discharge_type}_discharge_pct
#
#                # if discharge_pct_name in df.columns:
#                    # apply pct to total discharge
#                # else:
#                    # apply pct from dictionary
#            #
#
#    # TODO add in consumption and surface discharge


#    if total:
#        df = total_df
#    else:
#        df = output_df
#
#    return df

In [ ]:
for mine_type in fuel_water_type_dict['coal']:
    for water_type in fuel_water_type_dict['coal'][mine_type]:
        print(water_type)

In [ ]:
df.sum()

In [ ]:
df = flow.prep_electricity_generation()
df

In [ ]:
data = flow.configure_data()
sector_types = {'crop_irrigation': .15, 'golf_irrigation': .10}


df=flow.calc_water_conveyance_losses(data=data, sector_types=sector_types, total=True)
for col in df.columns:
    print(col)

In [ ]:
df = df.groupby('State', as_index=False).sum()
df

In [ ]:
data = flow.configure_data()
df = flow.calc_energy_agriculture(data=data, total=True)
for col in df.columns:
    print(col)

In [ ]:
df = flow.calc_wastewater_exports(data=data)
for col in df.columns:
    print(col)

In [ ]:
x.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\fuel_types.csv')

In [ ]:
name_list = ['electricity to rejected energy', 'energy discharge from sectoral use', 'energy in wastewater',
                      'energy in agriculture', 'energy in public water supply', 'energy production exports',
                     'water discharge from sectoral use']
name_list.index('electricity to rejected energy')

In [ ]:
water_discharge_dict = {'surfacewater': {'fresh': {'wastewater': 0, 'ocean': 0, 'surface': 1},
                                        'saline': {'wastewater': 0, 'ocean': 1, 'surface': 0}},
                        'groundwater': {'fresh': {'wastewater': 0, 'ocean': 0, 'surface': 1},
                                        'saline': {'wastewater': 0, 'ocean': 1, 'surface': 0}},
                                'pws': {'fresh': {'wastewater': 1, 'ocean': 0, 'surface': 0}}
                             }

In [ ]:
sector_consumption_dict = {'residential': {'groundwater': {'saline': 0, 'fresh': .3},
                                                   'surfacewater': {'saline': 0, 'fresh': .3},
                                                   'pws': {'fresh': .3}},
                            'commercial': {'groundwater': {'saline': 0, 'fresh': .15},
                                           'surfacewater': {'saline': 0, 'fresh': .15},
                                           'pws': {'fresh': .15}},
                            'industrial':{'groundwater': {'saline': .003, 'fresh': .15},
                                          'surfacewater': {'saline': .003, 'fresh': .15},
                                          'pws': {'fresh': .15}},
                            'mining': {'groundwater': {'saline': .03, 'fresh': .15},
                                       'surfacewater': {'saline': .03, 'fresh': .15}},
                            'crop_irrigation': {'groundwater': {'saline': 0, 'fresh': .3},
                                                'surfacewater': {'saline': 0, 'fresh': .3}},
                            'livestock':  {'groundwater': {'saline': 0, 'fresh': .87},
                                           'surfacewater': {'saline': 0, 'fresh': .87}},
                            'aquaculture': {'groundwater': {'saline': 0, 'fresh': .5},
                                            'surfacewater': {'saline': 0, 'fresh': .5}}}

In [ ]:
df = data.copy()

In [ ]:
saline_groundwater_industrial_mgd                                                                   42.9
saline_surfacewater_industrial_mgd                                                                742.92

In [ ]:
df = flow.calc_sectoral_use_water_discharge(data=data, total=True)

In [ ]:
x=16987.779934 +  6225.87913 + 4263.081345
x

In [ ]:
df1=flow.calc_sectoral_use_water_discharge(data=data, total=True)
df1 = df1[['FIPS', 'residential_wastewater_discharge_mgd', 'commercial_wastewater_discharge_mgd', 'industrial_wastewater_discharge_mgd']]

In [ ]:
df2 = flow.prep_wastewater_data()
df2 = df2[['FIPS', 'State', 'County', 'municipal_wastewater_mgd', 'infiltration_wastewater_mgd']]

In [ ]:
df3 = pd.merge(df2, df1, how='left')
df3['sector_total'] = (df3['residential_wastewater_discharge_mgd'] 
                                                 + df3['commercial_wastewater_discharge_mgd'] 
                                                 + df3['industrial_wastewater_discharge_mgd'])
df3['diff'] = df3['municipal_wastewater_mgd'] - df3['sector_total']
df3.sum()

In [ ]:
df = pd.DataFrame()
df_sector_discharge = flow.calc_sectoral_use_water_discharge(data=data, total=True)
df_sector_discharge = df_sector_discharge[['FIPS', 'State', 'County',
                                              'residential_wastewater_discharge_mgd',
                                              'commercial_wastewater_discharge_mgd',
                                              'industrial_wastewater_discharge_mgd']]



df['total_wastewater_discharge'] = df_sector_discharge[['residential_wastewater_discharge_mgd',
                                              'commercial_wastewater_discharge_mgd',
                                              'industrial_wastewater_discharge_mgd']].sum(axis=1)

df['wastewater_exports_mgd'] = data['municipal_wastewater_mgd'] - df['total_wastewater_discharge']
df

In [ ]:
df1 = flow.calc_electricity_rejected_energy(data=data)
df2 = flow.calc_sectoral_use_energy_discharge(data=data)
df3 = flow.calc_energy_wastewater(data=data)
df4 = flow.calc_energy_agriculture(data=data)
df5 = flow.calc_energy_pws(data=data)
df6 = flow.calc_energy_production_exports(data=data)
df7 = flow.calc_sectoral_use_water_discharge(data=data)
df_list = [df1, df2, df3, df4, df5, df6, df7]
df_list

In [ ]:
df = flow.calc_sectoral_use_water_discharge(data=data)
for col in df.columns:
    print(col)

In [ ]:
df2 = df[['residential_saline_groundwater_consumption_mgd',
'residential_fresh_groundwater_consumption_mgd',
'residential_saline_surfacewater_consumption_mgd',
'residential_fresh_surfacewater_consumption_mgd',
'residential_fresh_pws_consumption_mgd',]].sum()
#df[['residential_fresh_pws_consumption_mgd']].sum()
df2

In [ ]:
x = 733+17+6309
x

In [ ]:
df = flow.calc_sectoral_use_water_discharge(data=data, total=True)
df.sum()

In [ ]:
df = flow.prep_consumption_fraction()
df.sum()

In [ ]:
df = flow.prep_water_use_2015(variables=['County','fresh_surfacewater_residential_mgd', 
                                         'fresh_groundwater_residential_mgd', 'fresh_pws_residential_mgd']).sum(axis=0)
df

In [ ]:
df = flow.calc_energy_wastewater(data=data, total=True)
#df = df[df.State == "TX"]
for col in df.columns:
    print(col)

In [ ]:
total_df

In [ ]:
#test_dict.update({'test_zero':x})
test_dict
rejected_energy_total = pd.DataFrame.from_dict(test_dict, orient='index').transpose().sum(axis=1)
rejected_energy_total



In [ ]:
x = np.zeros((len(df), 1))
test_dict.update({'test_zero': x})
test_dict['test_zero']

In [ ]:
# Move this to top

    # calculate rejected energy and energy services for each fuel, agriculture, water, and pumping type
    for water_type in water_types_list:
        for pumping_type in pumping_type_dict:
            for agriculture_type in agriculture_type_list:
                for fuel_type in fuel_type_dict:
                    fuel_type_efficiency = 'agriculture_' + fuel_type + "_efficiency_pct"
                    if f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu' in energy_value_dict:
                        rejected_energy_name = f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_rejected_energy_bbtu'
                        energy_services_name = f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_energy_services_bbtu'
                        if fuel_type_efficiency in df.columns:
                            #df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_rejected_energy_bbtu'] = (1 - df[fuel_type_efficiency]) \
                            #                                                                                         * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']
                            #df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_energy_services_bbtu'] = (df[fuel_type_efficiency]) \
                            #                                                                                         * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']

                            rejected_energy_value = (1 - df[fuel_type_efficiency]) * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']
                            energy_services_value = (df[fuel_type_efficiency]) * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']

                            rejected_energy_dict.update({rejected_energy_name: rejected_energy_value})
                            energy_services_dict.update({energy_services_name: energy_services_value})

                        # if there's no fuel efficiency column, use defaults
                        else:
                            #df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_rejected_energy_bbtu'] = (1 -fuel_type_dict[fuel_type]) \
                            #                                                                             * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']
                            #df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_energy_services_bbtu'] = (fuel_type_dict[fuel_type]) \
                            #                                                                             * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']

                            rejected_energy_value = (1 -fuel_type_dict[fuel_type]) * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']
                            energy_services_value = (fuel_type_dict[fuel_type]) * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']

                            rejected_energy_dict.update({rejected_energy_name: rejected_energy_value})
                            energy_services_dict.update({energy_services_name: energy_services_value})

                        #df[f'{agriculture_type}_total_rejected_energy_bbtu'] = df[f'{agriculture_type}_total_rejected_energy_bbtu'] + df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_rejected_energy_bbtu']
                        #df[f'{agriculture_type}_total_energy_services_bbtu'] = df[f'{agriculture_type}_total_energy_services_bbtu'] + df[ f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_energy_services_bbtu']
                    else:
                        pass

In [ ]:
df= df[df.State == "CA"]
df = df['electricity_irrigation_ibt_rejected_energy_bbtu']
df.sum()

In [ ]:
df = flow.calc_energy_agriculture(data=data, total=True)
df

In [ ]:
test_dict = {"Gfg" : [4, 7, 5]}
  
# printing original dictionary
print("The original dictionary is : " + str(test_dict))
  
# initializing K 
K = 1
  
# one liner, values() getting all value according to keys
res = [sub[K] for sub in test_dict.values()]
res

In [ ]:
df = flow.calc_energy_agriculture(data=data, total=True)
df
#df.groupby("State", as_index=False).sum()

In [ ]:
df = flow.prep_pumping_intensity_data()
df

In [ ]:
df = flow.prep_irrigation_fuel_data()
df

In [ ]:
df = flow.calc_electricity_wastewater(data=data)
df

In [ ]:
df = flow.calc_sectoral_use_energy_discharge(data=data, total=True)
df

In [ ]:
data.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\run4.csv')

To override the data inputs, flow.configure_data() can be run.
If run with no arguments provided, this function aggregates all baseline data to calculate flow values for the united states in 2015. Parameters are set, however, such that this function can be used to swap out datafiles. 

For example, if alternative US county level data was available for interbasin transfer electricity use, that could be used instead of the default data.

In [ ]:
df = flow.configure_data()
df

In [ ]:
is_NaN = data.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = data[row_has_NaN]
rows_with_NaN

In [ ]:
df2.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\run2.csv')

<h1> Public Water Supply </h1>

## Water in Public Water Supply

### Public Water Supply Withdrawals

The USGS 2015 dataset provides public water supply withdrawals directly under data codes 'PS-WGWFr', 'PS-WSWFr','PS-WGWSa', 'PS-WSWSa' for each county.

In [ ]:
df = flow.prep_water_use_2015(variables=['State','County','PS-WGWFr', 'PS-WSWFr','PS-WGWSa', 'PS-WSWSa'])
df.head()               

### Public Water Supply Deliveries to Sectors, Exports, and Imports

The USGS 2015 dataset provide public water supply data in three categories: (1) total public water supply withdrawals, (2) public water supply deliveries to domestic end users, and (3) public water supply deliveries to thermoelectric cooling.

The 1995 USGS dataset provides public water supply deliveries to domestic, industrial, commercial, thermoelectric cooling, and losses/exports separately. To estimate the deliveries to commercial and industrial customers in 2015, the ratio of each to the sum of deliveries to residential and thermoelectric cooling is calculated from the 1995 data. These ratios are applied to the 2015 residential and thermoelectric cooling public water supply delivery values. 

Additionally, public water suppy exports and imports are calculated. When the sum of water deliveries is greater than the water withdrawn for public water supply, the difference is considered to be imported. When water withdrawn is greater, excess water after deliveries is considered to be exported.

Caveats and assumptions:
* This methodology assumes that if no public water supply was delivered to residential or thermoelectric cooling in 1995, then deliveries to industrial and commercial sectors would likewise be zero.

Potential mods
 for import pws counties, take 15% of withdrawals and imports as losses and add it on to the top of imports
 for export pws counties, take 15% of withdrawals as losses, take it out of exports, any remaining put as imports

In [ ]:
df = flow.calc_pws_discharge()
df.head()

## Energy in Public Water Supply

Energy in public water supply includes
groundwater pumping, surface water pumping, desalination (when applicable), surface water treatment, groundwater treatment, distribution, and interbasin transfers (when appropriate).

An assumption is made that water that goes through desalination additionally requires surface water or groundwater treatment

The same groundwater and surface water pumping intensities are used from agriculture sector.

Note that interbasin transfer data that is provided to this function is already in bbtu, not in water flows. 
For non-ibt data, the option is provided for the user to use region-level energy intensity values (as in groundwater and surface water pumping). When values are not available, assumptions for each in kWh/MG are set as default values that can be overriden.

intensity values are used from tidwell and LLNL

In [ ]:
df = flow.calc_electricity_public_water_supply(total=True)
df.head()

# Wastewater Treatment
Wastewater treatment facility data is provided by EPA for the year 2012. These values are assumed for 2015. Values are provided on a treatment plant level with data on the following variables: 
Flow:
    * Total water flows (MGD) treated by the facility
    * Total infiltration flows treated by the facility (leaks into pipes and conduits)
Treatment:
    * Treatment type of each treatment plant (primary, secondary, advanced)
Discharge:
    * discharge location of each plant, broken down by percent of flow to each discharge location (e.g., ocean discharge, suface discharge, irrigation reuse)
Location:
    * county information is included for each power plant
    
This data allows us to determine total water flows processed and discharged by wastewater facilities at a county level by treatment type. The treatment type allows us to calculate the electricity use by county for wastewater treatment,

## Water in Wastewater Treatment

### Water Inflows to Wastewater Treatment Facilities
Total flows processed by wastewater treatment facilities are provided in the EPA dataset directly. However, this data will have discrepencies from the water data delivered to various sectors. To estimate water flows from residential, commercial, and industrial sectors to wastewater treatment facilities, an assumption is made that all water delivered from public water supply to these sectors is sent to wastewater treatment facilities. This assumption is also used to fill wastewater treatment flow values for regions that are unavailable in the EPA dataset. For example, south carolina does not have wastewater treatment facility values in the EPA dataset. It is assumed, therefore, that total wastewater treatment flows in south carolina and equal to the total public water supply flows for each county and is attributed to municipal discharge rather than infiltration. It is assumed that 80% of treatment flows are advanced treatment and 20% are secondary treatment. It is also assumed that all discharge is sent to surface. This follows average values for the state of North Carolina which were used as a proxy in this instance.

### Quantity of Wastewater Processed by Treatment Type
Wastewater treatment plant treatment type is provided in the EPA data directly. These values are binned for simplification into no treatment, primary, secondary, and advanced treatment. When no treatment type was provided for a facility, it was assumed that they were secondary treatment facilities.

To see the amount of water (MGD) processed within each county by treatment type, run the cell below.

In [ ]:
df = flow.prep_wastewater_data()
df.head()

### Wastewater Treatment Facility Discharge

Wastewater treatment plant discharge location is provided in the EPA data directly. These values are binned for simplification into the following categories:

* 'outfall to surface waters': 'ww_sd',
* 'ocean discharge': 'ww_od',
* 'deep well': 'ww_gd',
* "reuse: industrial": 'ww_in',
* 'evaporation': 'ww_evap',
* 'spray irrigation': 'ww_ir',
* 'overland flow no discharge': 'ww_ww',
* 'overland flow with discharge': 'ww_sd',
* 'discharge to another facility': 'ww_ww',
* 'combined sewer overflow (cso) discharge': 'ww_sd',
* 'other': 'ww_sd',
* 'discharge to groundwater': 'ww_gd',
* 'no discharge, unknown': 'ww_ww',
* 'reuse: irrigation': 'ww_ir',
* 'reuse: other non-potable': 'ww_sd',
* 'reuse: indirect potable': 'ww_sd',
* 'reuse: potable': 'ww_ps',
* 'reuse: groundwater recharge': 'ww_gd'

In [ ]:
df = flow.prep_wastewater_data()
df = df[['FIPS','ww_evap','ww_gd','ww_in','ww_ir','ww_od','ww_ps','ww_sd','ww_ww']]
df.head()

## Energy in Wastewater Treatment

Wastewater treatment facilities have various levels of treatment from primary to advanced. Each treatment level has an increasing level of energy intensity to process the water. Intensity value estimates for wastwater treatment were used from Pabi et al. [17].

# Agriculture

## Water in Agriculture

## Energy in Agriculture

Energy use in agriculture consists of groundwater pumping, surface water pumping, and interbasin transers. USDA's Farm and Ranch Irrigation Survey (FRIS) provides state-by-state data on irrigation groundwater depth and average irrigation pressurization levels for irrigation within a state, enabling the calculation of pump electricity consumption for both groundwater and surface water pumping [13]. The 2013 survey is the closest year available to 2015 values. It is assumed that values do not vary significantly between the two years.

The methodology for calculating groundwater and surface water pumping energy is described in Tidwell et al [14]. However, where that publication uses well depth to water to calculate total differential height, the total well depth is used here as a way to offset some of the losses due to friction that would occur in the piping, as described in Lawrence Berkeley National Laboratory (LBNL) Home Energy Saver & Score: Engineering Documentation [15].

Pump efficiency is assumed to be the average (46.5%) of the range (34-59%) listed in [14]. State-level intensity rates are calculated here and applied to the county level water in agriculture values. The general methodology implemented here to determine total energy in agriculture is adapted from [5], not including the additions previously mentioned.

### Groundwater & Surface Water Pumping

### Interbasin Transfers

# Electricity Generation

## Energy in Electricity Generation

US Energy Information Administration (EIA) 923 data for 2015 is used to determine electricity generation by generator type within each interconnect [12]. The datafile contains information on generation at the plant-level. EIA additionally provides data on which North American Electric Reliability Corporation (NERC) balancing region each plant belongs to. By mapping each NERC region to its associated interconnect, generation within each interconnect can be determined directly.

From EIA, the AER Fuel Type Code is used to determine generator type. EIA provides generator type information at a high level of detail. For this reason different generator types are grouped for simplicity using a generator key (GenType923Key.xlsx). For example, different types of natural gas generators are all grouped into "Natural Gas" as a type.

Energy production is provided as "Net Generation (Megawatthours)" in the EIA dataset, "Total Fuel Consumption (MMBTU)" is the MMBTU equivalent of fuel going to each plant for electricity generation. The difference between these two values indicates the amount of energy generation that does not go to energy losses.

## will be filled in with calculate function

## Water in Electricity Generation

# End Use Sectors

## Residential Sector

### Water in Residential Sector

### Energy in Residential Sector

#### Energy (electricity) Demand in Residential Sector

##### weighted by population. <b> consider splitting out by usage if data is available

In [ ]:
df = flow.prep_electricity_demand_data()
df

#### Energy (fuel) Demand in Residential Sector

## Commercial Sector

### Water in Commercial Sector

### Energy in Commercial Sector

## Industrial Sector

### Water in Industrial Sector

### Energy in Industrial Sector

## Transportation Sector

### Energy in Energy Sector

# Energy Production

## Biomass

### Energy in Biomass Production (Ethanol)

To find the ethanol production per county, this calculation takes 2015 eia data on ethanol plant capacity which has added locational data by each plant and combines with state level biomass (ethanol) production from prep_state_fuel_production_data() to split out state total by county. Returns a dataframe of ethanol production (bbtu) by county FIPS for each county in the US for 2015.

ethanol plant production filler for wyoming (in state total but not location data) ,Torrington, WY100%, https://neo.ne.gov/programs/stats/122/2015/122_201512.htm

#### <b> <ul> <font color='red'> Consider looking for estimates for electricity use in biomass production to split out from industrial

### Water in Biomass Production (Ethanol)

#### Water used in irrigation of corn used for ethanol

Data on water used in the irrigation of corn ultimately used for ethanol production does not exist explicitly at any level of granularity and must be calculated out from larger state-level corn growth irrigation intensity estimates.

The percent of corn grown for ethanol in the US was used from US DOE Alternative Fuels Data Center [21]. Total corn production for ethanol (5.22 billion bushels) was divided by total corn producton (13.6 billion bushels) for 2015 and calculated to be 38.406%. Data on a state or county-level does not exist, therefore, this percentage is applied to all states with corn production. A state-level fraction of corn used for ethanol production could be calculated from the amount of ethanol produced in a state and applying an assumption of corn required per unit of ethanol, however, not all corn used in ethanol is produced in the state that uses it which lead to states such as CA with an ethanol fraction of corn growth of nearly 900% of their total corn production. Given that we are unable to determine which states the imported corn for ethanol would be coming from, the decision was made to apply the US total fraction to all states.

Total water used in the production of corn for ethanol is therefore the assumed 38.406% multiplied by the product of the total amount of irrigated acres of corn (AC_Irr_Corn) and the calculated irrigation intensity for that state (total acres applied to all crops / total acres irrigated, all crops) provided from USDA Census of Agriculture Irrigation and Water Management Survey [22], Table 35 and Table 4, respectively. The irrigation intensity for corn is calculated from the average irrigation intensity for the state.

To split the total MGY applied between fresh surface and groundwater, USDA FRIS Census of Agriculture data on the fraction of surface water (surface water + off farm water) from total water supply (including groundwater) in corn production is used from Table 37 of [22]. Off-farm sources are assumed to be surface water.

Some states that grow corn do not provide data on the water source breakdown (surface vs. ground) for their irrigation. The surface to groundwater ratio for all irrigation in a state is substituted for these states. These values are calculated from the USGS 2015 water dataset.

In [ ]:
df = flow.prep_county_water_corn_biomass_data()

#### Water used in the production of ethanol (fermentation)

#### <b> <ul> <font color='red'> Consider looking for estimates for water used in the fermentation process of ethanol production, not just corn growth.

## Petroleum

### Energy in Petroleum Production (Conventional and Unconventional Oil Drilling)

 crude oil production by county fillers
Idaho: https://ogcc.idaho.gov/wp-content/uploads/sites/3/2017/06/2016-10-6-state-of-idaho-releases-oil-gas-production-data.pdf
100% of Idaho petroleum and natural gas is from Payette County

Alaska,
https://www.commerce.alaska.gov/web/aogcc/Data.aspx
almost all oil is produced along the north slope. Calculated from oil production download. The rest is on the kenai peninsula (cook inlet basin)

Maryland
https://www.eia.gov/state/analysis.php?sid=MD#34
assumed 50/50 split between Garret and Allegany Counties

Nevada
https://gisweb.unr.edu/OilGas/
Assumed to be in nye county with producing oil wells (only 4 btu)

Oregon
https://www.oregongeology.org/mlrr/oilgas-report.htm
Only one county (columbia county)

### Water in Petroleum Production (Conventional and Unconventional Oil Drilling)

## Natural Gas

### Energy in Natural Gas Production (Unconventional Drilling)

### Water in Natural Gas Production (Unconventional Drilling)

## Coal

### Energy in Coal Production (Coal Mining)

### Water in Coal Production (Coal Mining)

# Intensities & Percentages

#### Conveyance Losses as a Percent of Water Withdrawn for Irrigation
Conveyance loss fraction for irrigation is calculated from the USGS 1995 water use dataset. It is calculated as the total conveyance losses (MGD) divided by the total water withdrawn (MGD) for irrigation. Looking at the fractions at the county level shows that a small subset of counties have unreliable values where conveyance losses are far greater than the amount withdrawn for irrigation. As a result, the ability to cap the loss fraction at a specified amount it provided and replace it with the average of the specified region. The default loss cap fraction is .90.

In [ ]:
df = flow.prep_water_use_1995()
df = flow.calc_conveyance_loss_frac(df)
df.head()

##### Change in loss cap

In [ ]:
df = flow.prep_water_use_1995()
df = flow.calc_conveyance_loss_frac(df, loss_cap_amt=.20)
df.head()

In [ ]:
help(flow.plot_bar)

In [ ]:
flow.plot_bar(df, "State","IR_CLoss_Frac", "State", y_axis_title = "Irrigation Conveyance Loss Fraction", x_axis_title="State")

#### Water Use Intensity for Hydroelectric Generation

In [ ]:
df = flow.calc_hydroelectric_water_intensity(intensity_cap=True, intensity_cap_amt=165, region_avg=False)

df.head()

In [ ]:
flow.plot_bar(df, "State","HY_IF", "State")